In [ ]:
import re
import json
import time
from IPython.display import Markdown

def read_msg(n=None, relative=False, id=None):
    """Stub function - replace with actual implementation"""
    if id:
        return {'content': 'your_target_code_here'}
    return {'id': 'msg_123'}

def Chat(model):
    """Stub function - replace with actual implementation"""
    class MockChat:
        def __call__(self, prompt):
            class MockResponse:
                class MockChoice:
                    class MockMessage:
                        content = '```python\n# needed imports\nimport os\n\nyour_target_code_here\n```'
                    message = MockMessage()
                choices = [MockChoice()]
            return MockResponse()
    return MockChat()

def curr_dialog():
    """Stub function - replace with actual implementation"""
    return {'name': 'dialogs/current_dialog'}

def create(filename, content):
    """Stub function - replace with actual implementation"""
    with open(filename, 'w') as f:
        f.write(content)


In [ ]:
def isolate(ids=None, path=None, suffix=None):
    "Create isolated dialog with target message(s) and auto-detected dependencies"
    if ids is None: ids = read_msg(n=-1, relative=True)['id']
    if isinstance(ids, str): ids = [ids]
    target_code = read_msg(0, id=ids[0])['content']
    chat_inst = Chat('claude-sonnet-4-5')
    resp = chat_inst(f'Add only the minimal code needed before this to make it run:\n```python\n{target_code}\n```')
    resp_text = resp.choices[0].message.content
    prefix_code = re.search(r'```python\n(.*?)\n```', resp_text, re.DOTALL).group(1)
    idx = prefix_code.find(target_code)
    needed_prefix = prefix_code[:idx].strip() if idx != -1 else prefix_code
    cells = [{'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': [f"{line}\n" for line in needed_prefix.splitlines()]}]
    cells.append({'cell_type': 'code', 'execution_count': None, 'metadata': {}, 'outputs': [], 'source': [f"{line}\n" for line in target_code.splitlines()]})
    nb = dict(cells=cells, metadata=dict(kernelspec=dict(display_name='python3', language='python', name='python3')), nbformat=4, nbformat_minor=4)
    dlg_name = curr_dialog()['name']
    isolated_stem = f"{dlg_name.split('/')[-1]}_isolated_{int(time.time())}"
    isolated_file = f"{isolated_stem}.ipynb"
    isolated_url = f"/dialog_?name={dlg_name.rsplit('/', 1)[0]}/{isolated_stem}"
    create(isolated_file, json.dumps(nb, indent=2))
    return Markdown(f'<a href="{isolated_url}" target="_blank">Open isolated dialog</a>')
